### $\color{purple}{\text{Missing Data in the Age of Machine Learning and Artifical Neural Network (Part 3)}}$

## $\color{purple}{\text{Imputation in Machine Learning and Multiple Imputation}}$

## $\color{purple}{\text{How Imputation Fits Into Your Machine Learning Models}}$

### Typical Machine Learning Workflow: 
  * Encoding
  * Preprocessing
  * Train
  * Test
  * Use
  
![](https://raw.githubusercontent.com/WestHealth/pydataglobal-2022/main/images/typical.svg)


### Machine Learning Workflow with Imputation: 

  * Encoding
  * Preprocessing
  * *Imputater*
  * Train
  * Test
  * Use


![](https://raw.githubusercontent.com/WestHealth/pydataglobal-2022/main/images/imputer.svg)


* We demonstrate this using `sklearn`'s pipeline. But this is meant to describe abstractly what you should do

Same data set is taken from the [Wine Quality Dataset at UCI](https://archive.ics.uci.edu/ml/datasets/Wine+Quality)

This demonstrates a typical pipeline. The final column `quality` is the predicted value (ranging from 1 - 9). The `features` variable contains all the other column names

### $\color{purple}{\text{Libraries for this lesson}}$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from autoimpute.imputations import MiceImputer
from autoimpute.imputations import SingleImputer
from matplotlib.patches import Rectangle
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
training = pd.read_csv('data/original_wine_training.csv')
test = pd.read_csv('data/original_wine_test.csv')

In [ ]:
training

In [ ]:
features = list(training.columns[0:-1])
features

### Typical ML Workflow

We build the pipeline by one-hot encoding the `type` column which is categorical, then scale it, then apply random forest regressor

In [ ]:
pipeline = make_pipeline(
    ColumnTransformer([("type", OneHotEncoder(), ["type"])],remainder='passthrough'), 
    StandardScaler(),
    RandomForestRegressor())

In [ ]:
pipeline.fit(training[features], training['quality'])

In [ ]:
pipeline.score(test[features], test['quality'])

### ML Workflow with Imputer

Note: This is meant to demonstrate workflow and `autoimpute` is used as an example.

One drawback is that `autoimpute` imputers require a `pandas` `DataFrame` as an input so custom transformers need to be used.


In [ ]:
pandas_hack = FunctionTransformer(
    lambda x: pd.DataFrame(x, columns=['type_r', 'type_w'] + features[0:-1]))
pandas_hack_full = FunctionTransformer(
    lambda x: pd.DataFrame(x, columns=['type_r', 'type_w'] + features))

We can insert the imputer into the pipeline

In [ ]:
pipeline = make_pipeline(
    ColumnTransformer([("type", OneHotEncoder(), ['type'])],remainder='passthrough'), 
    pandas_hack,
    SingleImputer(strategy='least squares'), 
    StandardScaler(),
    RandomForestRegressor())

In [ ]:
wine_training = pd.read_csv('data/wine_training.csv')
pipeline.fit(wine_training[features], wine_training['quality'])

In [ ]:
wine_test = pd.read_csv('data/wine_test.csv')
pipeline.score(wine_test[features], wine_test['quality'])

In [ ]:
wine_future = pd.read_csv('data/wine_future.csv')
pipeline.predict(wine_future[features].iloc[0:10])

## $\color{purple}{\text{How does Multiple Imputation fit in?}}$
### $\color{purple}{\text{Approach 1: Augment Data with Multiple Copies}}$

![](https://raw.githubusercontent.com/WestHealth/pydataglobal-2022/main/images/stack.svg)

Augmentation teachs the model that the imputed values are "fuzzy" by providing different values.

We create the same pipeline except we have a MiceImputer at the end.
The resultant `dfs` are 5 copies of our dataframe with 5 separate imputations

In [ ]:
pipeline = make_pipeline(
    ColumnTransformer([("type", OneHotEncoder(), ['type'])],remainder='passthrough'), 
    StandardScaler(), 
    pandas_hack,
    MiceImputer(k=5, strategy='stochastic'))

dfs = [each[1] for each in pipeline.fit_transform(wine_training[features])]
print('Number of dataframe: ' + str(len(dfs)))

dfs[0]

We augment the training set by concatenating the 5 different data frame. Equivalently, we could rotate each epoch with different imputations.

In [ ]:
augmented_training = pd.concat(dfs)

quality = pd.concat([
    wine_training.quality, wine_training.quality, wine_training.quality,
    wine_training.quality, wine_training.quality
])

In [ ]:
augmented_training

Since test set is used to evaluate when the model may hit overtraining, it is not necessary to multiply impute the tests. But you may.

In [ ]:
test_dfs = [each[1] for each in pipeline.transform(wine_test[features])]

test1 = test_dfs[0]  # Variation one, just take one imputation
quality1 = wine_test.quality

test2 = pd.concat(test_dfs)  # Variation two augment in the same way
quality2 = pd.concat([quality1, quality1, quality1, quality1, quality1])

In [ ]:
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor())

In [ ]:
pipeline.fit(augmented_training, quality)

In [ ]:
pipeline.score(test1, quality1)

In [ ]:
pipeline.score(test2, quality2)

Here we show an additional example of using neural networks to predict the imputed dataset. The network model was built as a classification problem. Bear in mind this is just for demonstration purposes, our model is not a particularly good estimator

In [ ]:
# Neural Network Approach
model = tf.keras.models.Sequential([
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation=tf.nn.tanh),
    tf.keras.layers.Dense(20, activation=tf.nn.tanh),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(augmented_training, quality, epochs=10, verbose=False)

In [ ]:
model.evaluate(test1, quality1)

In [ ]:
model.evaluate(test2, quality2)

#### What about future values?
Same options:
 * take one imputation
 * run all imputations through the model and use an ensemble technique to combine (e.g., majority voting)

In [ ]:
pipeline = make_pipeline(
    ColumnTransformer([("type", OneHotEncoder(), ['type'])],remainder='passthrough'), 
    StandardScaler(), 
    pandas_hack,
    MiceImputer(k=5, strategy='stochastic'))

future_dfs = [
    each[1].iloc[[0]] for each in pipeline.fit_transform(wine_future[features])
]

future1 = future_dfs[0]  # Variation one, just take one imputation
future2 = pd.concat(future_dfs)  # Variation two augment in the same way

In [ ]:
model.predict(future1)

In [ ]:
# Variation 1: Pick First Imputed
# Argmax was used because the model predicted one-hot encoded label (i.e. 000000100 for 7)

np.argmax(model.predict(future1))

It is worthy to mention that here we simply take the sum to aggregate 5 imputation results. In practice, you can use other more robusted methods such as majority vote, averaging, etc. 

In [ ]:
# Variation 2: Aggregate all Imputed
np.argmax(model.predict(future2).sum(axis=0))

### $\color{purple}{\text{Approach 2: Combine Multiple Models}}$

![](https://raw.githubusercontent.com/WestHealth/pydataglobal-2022/main/images/ensemble.svg)

In [ ]:
models = [
    tf.keras.models.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation=tf.nn.tanh),
        tf.keras.layers.Dense(20, activation=tf.nn.tanh),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ]) for _ in range(0, 5)
]

for model in models:
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

Each model is trained on a different imputation model. Each model should be tested under each imputation model

In [ ]:
# Reminder: dfs stores a list of 5 imputed dataframes that was done indepdently

for model, training in zip(models, dfs):
    model.fit(training, wine_training.quality, epochs=10, verbose=False)

In [ ]:
for model, test in zip(models, test_dfs):
    print(model.evaluate(test, wine_test.quality, verbose=False))

In [ ]:
# np vstack turns the list of arrays into an array of arrays
predictions = np.vstack(
    [model.predict(future) for model, future in zip(models, future_dfs)])

In [ ]:
# Aggregate predictions
np.argmax(predictions.sum(axis=0))

### $\color{purple}{\text{Worth Mentioning: Use Single Imputation with Bagging}}$
Short for bootstrap and aggregation

Rather than multiple imputation, single imputations are performed from resampled datasets (bootstrapping)

![](https://raw.githubusercontent.com/WestHealth/pydataglobal-2022/main/images/single.svg)

In [ ]:
pipeline = make_pipeline(
    ColumnTransformer([("type", OneHotEncoder(), ['type'])],remainder='passthrough'), 
    StandardScaler(),
    pandas_hack_full, 
    MiceImputer(k=1, strategy='stochastic')
)

bagged_dfs = [
    next(pipeline.fit_transform(wine_training.sample(frac=1, replace=True)))[1]
    for _ in range(0, 5)
]

In [ ]:
models = [
    tf.keras.models.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation=tf.nn.tanh),
        tf.keras.layers.Dense(20, activation=tf.nn.tanh),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ]) for _ in range(0, 5)
]

for model in models:
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
for model, training in zip(models, dfs):
    model.fit(training, wine_training.quality, epochs=10, verbose=False)

In [ ]:
predictions = np.vstack(
    [model.predict(future) for model, future in zip(models, future_dfs)])
# Aggregate predictions
np.argmax(predictions.sum(axis=0))

## $\color{purple}{\text{Conclusion}}$

* In a typical machine learning pipeline, imputation should be done before feeding into the machine learning model 

* Several strategies for multiple imputations:
  * Augmenting dataset multiple times and predicted by the same imputer 
  * Perform imputation multiple times with independent imputer
  * Bagging can be applied to single imputation performed multiply
